In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/covid19-indonesia/covid_19_indonesia_time_series_all.csv')
data

In [ ]:
import datetime as dt
data['Date'] = data.apply(lambda x: pd.to_datetime(x['Date']), axis = 1)
data
#Ukuran data 37 kolom dan 9259 baris

In [ ]:
data.columns

In [ ]:
#DATA CLEANSING
data = data[['Date','Location','New Cases','New Deaths','New Recovered','New Active Cases','Total Cases','Total Deaths','Total Recovered','Total Active Cases', 'Population Density']]
data

In [ ]:
data.info()   #sudah tidak ada nilai yang tidak valid, sehingga data bisa digunakan

In [ ]:
data.head() #Menunjukkan sampel data

In [ ]:
for x in data.columns :
    bawah = data.sort_values(x).iloc[0][x]
    atas = data.sort_values(x).iloc[-1][x]
    print('Data {} berada pada rentang {} sampai {}'.format(x, bawah, atas))

In [ ]:
data.describe(percentiles = [0.1,0.25,0.5,0.75,0.9])

In [ ]:
#Perbandingan kategori
#1. Kasus aktif yang terakhir tercatat di masing-masing provinsi
data[data['Location']!='Indonesia'].drop_duplicates(subset = 'Location', keep = 'last').plot(kind='bar', y = 'Total Active Cases', x='Location', title = 'Banyaknya Kasus Aktif Untuk Masing-Masing Provinsi').set(xlabel='Provinsi',ylabel='Banyak Kasus Aktif')

In [ ]:
#Perbandingan kategori
#2. Kematian per hari maksimum tiap provinsi
grafik = data[data['Location']!='Indonesia'].sort_values('New Deaths', ascending = False).drop_duplicates(subset='Location', keep = 'first').plot(kind='barh', x='Location', y='New Deaths', title = 'Jumlah Kematian Dalam Satu Hari Terbanyak di Tiap Provinsi', figsize = (10,10))
grafik.set(xlabel='Kematian per Hari Maksimum', ylabel = 'Provinsi')


In [ ]:
#Perubahan total kasus terhadap waktu
data[data['Location']=='Indonesia'].plot(kind = 'line', x = 'Date', y = 'Total Active Cases', title = 'Total Kasus Aktif Seiring Berjalannya Waktu').set(xlabel = 'Tanggal', ylabel = 'Total Kasus Aktif')

In [ ]:
#Perubahan komposisi total pasien sembuh dan total pasien meninggal seiring waktu
grafik2 = data[data['Location']=='Indonesia'].plot(kind='area', x = 'Date', y = ['Total Recovered', 'Total Deaths'], title = 'Komposisi Pasien Sembuh dan Pasien Meninggal Seiring Waktu')
grafik2.set_yscale('symlog')
grafik2.set(xlabel = 'Tanggal', ylabel = 'Jumlah Orang')

In [ ]:
#BINGUNG BAGIAN PENAMPILAN HIERARKI

In [ ]:
#Plotting relationship
#1 Total kasus aktif terakhir terhadap kepadatan penduduk
data[data['Location']!='Indonesia'].sort_values('Date').drop_duplicates(subset = 'Location', keep = 'last').plot(kind='scatter', x = 'Population Density', y = 'Total Active Cases', title = 'Grafik Total Kasus Aktif terhadap Kepadatan Penduduk di Tiap Provinsi')

In [ ]:
#2 Hubungan kasus baru per hari terhadap jumlah kasus aktif
data[data['Location']=='Indonesia'].plot(kind = 'scatter', x = 'Total Active Cases', y = 'New Cases', title = 'Grafik Pertambahan Kasus per Hari terhadap Total Kasus Aktif')

In [ ]:
import seaborn as sns
#Heatmap untuk data Indonesia seiring waktu
corr_matrix = data[data['Location']=='Indonesia'].drop(['Location','Population Density'], axis=1).corr()
mask = np.zeros_like(corr_matrix, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr_matrix, mask = mask, annot = True)

In [ ]:
import matplotlib.pyplot as plt
#Dari heatmap di atas, terdapat kaitan sangat erat antara total kasus dengan total pasien yang sembuh. Coba kita visualisasikan
x = data[data['Location']=='Indonesia']['Total Cases']
y = data[data['Location']=='Indonesia']['Total Recovered']
koefisien =np.polyfit(x,y,1)
fungsi = np.poly1d(koefisien)
plt.plot(x,y,'.',label = 'data')
plt.plot(x,fungsi(x),'-',label = 'Gradien = {}'.format(koefisien[0]))
plt.legend(loc='upper left')
plt.xlabel('Total Kasus')
plt.ylabel('Total Pasien Sembuh')
plt.title('Grafik Total Pasien Sembuh Terhadap Total Kasus')
plt.show()

#Sehingga dapat disimpulkan rasio pasien sembuh terhadap total kasus adalah 84%

In [ ]:
#Juga terdapat hubungan erat antara kasus baru dengan total kasus aktif, sesuai prediksi teori epidemiologi
x = data[data['Location']=='Indonesia']['Total Active Cases']
y = data[data['Location']=='Indonesia']['New Cases']
koefisien =np.polyfit(x,y,1)
fungsi = np.poly1d(koefisien)
plt.plot(x,y,'.',label = 'data')
plt.plot(x,fungsi(x),'-',label = 'Gradien = {}'.format(koefisien[0]))
plt.legend(loc='upper left')
plt.xlabel('Total Kasus Aktif')
plt.ylabel('Kasus Baru per Hari')
plt.title('Grafik Kasus Baru per Hari Terhadap Total Kasus Aktif')
plt.show()
#Dapat diambil kesimpulan, setiap 100 orang penderita Covid-19 menularkan penyakitnya ke 7 orang baru

In [ ]:
#Heatmap untuk data tiap provinsi di satu waktu tertentu (data terakhir yang diambil) Khusus untuk mengamati hubungan kepadatan penduduk dan variabel lainnya
corr_matrix = data[data['Location']!='Indonesia'].sort_values('Date').drop_duplicates(subset='Location', keep = 'last').drop(['Location'], axis = 1).corr()
mask = np.zeros_like(corr_matrix, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr_matrix, mask = mask, annot = True)

In [ ]:
#Yang menjadi perhatian adalah hubungan kepadatan penduduk dengan variabel lain
#Terdapat hubungan yang erat antara kepadatan penduduk dengan total kasus di suatu provinsi
x = data[data['Location']!='Indonesia'].sort_values('Date').drop_duplicates(subset='Location', keep = 'last')['Population Density']
y = data[data['Location']!='Indonesia'].sort_values('Date').drop_duplicates(subset='Location', keep = 'last')['Total Cases']
koefisien =np.polyfit(x,y,1)
fungsi = np.poly1d(koefisien)
plt.plot(x,y,'.',label = 'data')
plt.plot(x,fungsi(x),'-',label = 'Gradien = {}'.format(koefisien[0]))
plt.legend(loc='upper left')
plt.xlabel('Kepadatan Penduduk')
plt.ylabel('Total Kasus')
plt.title('Grafik Total Kasus Terhadap Kepadatan Penduduk Suatu Provinsi')
plt.show()
#Dapat ditarik kesimpulan, makin tinggi kepadatan penduduk, makin tinggi pula kasus Covid-19 di provinsi tersebut